## Traer audios

In [3]:
import os
import shutil
import logging
import pyodbc
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from collections import defaultdict

# ── CONFIGURACIÓN ───────────────────────────────────────────────────────────────
RUTA_ORIGEN       = r"\\SRVSFTPFIVE9\Users\sftpuser\recordings"
FECHA_OBJETIVO    = "12_25_2025"  # Fecha de carpeta (M_D_YYYY)
RUTA_DESTINO_BASE = r"C:\prueba_audios\16-09-2025_real"

# Conexión SQL Server
DB_SERVER   = "172.16.1.33"
DB_NAME     = "CUN_REPOSITORIO"

EXTENSIONES_AUDIO = ['.wav', '.mp3', '.wma']
# ────────────────────────────────────────────────────────────────────────────────

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)-8s - %(message)s',
    handlers=[
        logging.FileHandler('monitor_audio.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

def cargar_plantilla_sql():
    """Carga la plantilla desde SQL Server usando Trusted_Connection."""
    try:
        conn = pyodbc.connect(
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={DB_SERVER};"
            f"DATABASE={DB_NAME};"
            "Trusted_Connection=yes;"
        )

        query = """
        SELECT 
            Identificacion AS Identificación,
            Nombre AS Nombres,
            Apellidos AS Apellidos,
            NOM_CARG AS [Descripción Cargo],
            box_mail AS Correo
        FROM dbo.Planta_Activa
        WHERE NOM_CARG LIKE '%auxi%'
        """

        df = pd.read_sql(query, conn)
        conn.close()

        logging.info(f"✅ Consulta SQL ejecutada. Registros obtenidos: {len(df)}")

    except Exception as e:
        logging.error(f"Error al conectar con SQL Server: {e}")
        return {}

    if 'Correo' not in df.columns or 'Descripción Cargo' not in df.columns:
        logging.error("La consulta SQL no devuelve columnas 'Correo' y 'Descripción Cargo'.")
        return {}

    return dict(zip(df['Correo'].str.lower(), df['Descripción Cargo']))

def obtener_fecha_folder():
    if FECHA_OBJETIVO:
        mm, dd, yyyy = FECHA_OBJETIVO.split('_')
        fecha = datetime(int(yyyy), int(mm), int(dd))
    else:
        fecha = datetime.now() - timedelta(days=1)
    return f"{fecha.month}_{fecha.day}_{fecha.year}"

def crear_dir(ruta):
    Path(ruta).mkdir(parents=True, exist_ok=True)

def es_audio(nombre):
    return any(nombre.lower().endswith(ext) for ext in EXTENSIONES_AUDIO)

def limpiar_nombre(nombre_archivo):
    return nombre_archivo.replace('+', '')

def copiar(origen, destino_base, cargo, correo, fecha):
    destino_carpeta = os.path.join(destino_base, cargo, correo, fecha)
    crear_dir(destino_carpeta)

    nombre_archivo_limpio = limpiar_nombre(os.path.basename(origen))
    dst = os.path.join(destino_carpeta, nombre_archivo_limpio)

    if os.path.exists(dst) and os.path.getsize(dst) == os.path.getsize(origen):
        return False

    try:
        shutil.copy2(origen, dst)
        return True
    except Exception as ex:
        logging.error(f"Error al copiar '{origen}': {ex}")
        return False

# <<< CAMBIO: mapeo especial de carpetas “no-email”
ESPECIALES = {
    'cun',
    'vinculaciones ob',
    'vinculaciones ib',
}
CARGO_ESPECIAL = 'AUXILIAR GESTOR VINCULACIONES'
DOMINIO = '@cun.edu.co'

def resolver_correo_y_cargo(asesor_nombre: str, cargo_por_correo: dict) -> tuple[str, str]:
    """
    - Si la carpeta del asesor es una de las especiales, le agrega @cun.edu.co
      y le asigna el cargo AUXILIAR GESTOR VINCULACIONES.
    - En caso contrario, intenta buscar el cargo por correo en la plantilla SQL.
    """
    asesor_key = asesor_nombre.strip().lower()
    if asesor_key in ESPECIALES:
        correo = asesor_key + DOMINIO
        cargo = CARGO_ESPECIAL
        logging.debug(f"⚙️ Mapeo especial aplicado: {asesor_nombre} -> {correo} / {cargo}")
        return correo, cargo

    # Si ya viene con dominio, se usa tal cual; si no, se deja como está (tu lógica original)
    correo = asesor_key
    cargo = cargo_por_correo.get(correo, 'SIN_CARGO_DEFINIDO')
    return correo, cargo
# >>> FIN CAMBIO

def procesar_audios():
    FECHA_FOLDER = obtener_fecha_folder()
    RUTA_DESTINO = os.path.join(RUTA_DESTINO_BASE, f"audios {FECHA_FOLDER}")
    
    total_encontrados = 0
    total_copiados = 0
    conteo_por_cargo = defaultdict(int)

    # Cargar plantilla desde SQL
    CARGO_POR_CORREO = cargar_plantilla_sql()

    if not os.path.isdir(RUTA_ORIGEN):
        logging.error(f"No existe ruta origen: {RUTA_ORIGEN}")
        return

    for asesor in os.listdir(RUTA_ORIGEN):
        carpeta_asesor = os.path.join(RUTA_ORIGEN, asesor)
        if not os.path.isdir(carpeta_asesor):
            continue

        # <<< CAMBIO: resolver correo/cargo con la regla especial
        correo, cargo = resolver_correo_y_cargo(asesor, CARGO_POR_CORREO)
        # >>> FIN CAMBIO

        carpeta_fecha = os.path.join(carpeta_asesor, FECHA_FOLDER)
        if not os.path.isdir(carpeta_fecha):
            continue

        for root, _, files in os.walk(carpeta_fecha):
            for archivo in files:
                if not es_audio(archivo):
                    continue
                total_encontrados += 1
                ruta_arch = os.path.join(root, archivo)
                if copiar(ruta_arch, RUTA_DESTINO, cargo, correo, FECHA_FOLDER):
                    total_copiados += 1
                    conteo_por_cargo[cargo] += 1

    logging.info(f"🔍 Total archivos de audio encontrados: {total_encontrados}")
    logging.info(f"📦 Total archivos copiados: {total_copiados}")

    # Mostrar conteo por cargo
    logging.info("📊 Archivos copiados por cargo:")
    for cargo, cantidad in conteo_por_cargo.items():
        logging.info(f"   {cargo}: {cantidad}")

if __name__ == "__main__":
    procesar_audios()


C:\Users\haider_bello\AppData\Local\Temp\ipykernel_39220\2424318246.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-12-26 10:40:45,456 - INFO     - ✅ Consulta SQL ejecutada. Registros obtenidos: 253
2025-12-26 10:40:56,187 - INFO     - 🔍 Total archivos de audio encontrados: 3
2025-12-26 10:40:56,187 - INFO     - 📦 Total archivos copiados: 3
2025-12-26 10:40:56,203 - INFO     - 📊 Archivos copiados por cargo:
2025-12-26 10:40:56,204 - INFO     -    AUXILIAR GESTOR VINCULACIONES: 3


## Txt

In [ ]:

# !pip install faster-whisper torch --upgrade
import os
import time
import math
import threading
from queue import Queue
from pathlib import Path
from faster_whisper import WhisperModel
import torch

# ───── RUTAS (AJUSTA) ───────────────────────────────────────────────────────────
INPUT_ROOT = Path(r"C:\prueba_audios\16-09-2025_real\audios 12_22_2025\AUXILIAR GESTOR VINCULACIONES")
OUTPUT_DIR = Path(r"C:\descargas\cltiene_nov")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

AUDIO_EXTS = {".wav", ".mp3", ".m4a", ".flac", ".ogg", ".aac", ".wma"}

# ───── DETECCIÓN GPU / CONFIG ──────────────────────────────────────────────────
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Para 90% de uso necesitas GPU. No se detectó CUDA."
torch.set_float32_matmul_precision("medium")

MODEL_NAME   = "medium"
COMPUTE_TYPE = "large-v2"           # FP16 en GPU = rápido
BEAM_SIZE    = 5                   # 1 = más veloz
LANG         = "es"

# Estimar cuántas instancias caben en 12 GB (aprox medium FP16 ~4-5 GB)
free, total = torch.cuda.mem_get_info()
free_gb  = free / (1024**3)
total_gb = total / (1024**3)

# Máximo 3 modelos; si VRAM libre <7.5 GB, usa 2; si >=11 GB, prueba 3
MAX_MODELS = 3 if free_gb >= 11 else (2 if free_gb >= 7.5 else 1)
print(f"GPU VRAM libre: {free_gb:.1f} GB / {total_gb:.1f} GB → instancias: {MAX_MODELS}")

# ───── UTILIDADES ───────────────────────────────────────────────────────────────
def gather_audio_files(root: Path):
    for dp, _, files in os.walk(root):
        for f in files:
            if Path(f).suffix.lower() in AUDIO_EXTS:
                yield Path(dp) / f

def transcribe_with_model(model: WhisperModel, audio_path: Path, out_dir: Path) -> str:
    out_path = out_dir / (audio_path.stem + ".txt")
    if out_path.exists():
        return f"➡️ {audio_path.name} (ya existe)"
    segments, _ = model.transcribe(
        str(audio_path),
        language=LANG,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=400),
        beam_size=BEAM_SIZE,
        best_of=1,
        temperature=0.0,
        without_timestamps=True,
        word_timestamps=False,
        condition_on_previous_text=True,
    )
    text = "".join(s.text for s in segments).strip()
    out_path.write_text(text, encoding="utf-8")
    return f"✔️ {audio_path.name} ({len(text)} chars)"

# ───── POOL DE WORKERS (CADA UNO CON SU MODELO) ────────────────────────────────
def worker_loop(worker_id: int, q: Queue, results: list, lock: threading.Lock):
    # Cada worker carga SU propia instancia del modelo (mantiene GPU ocupada)
    model = WhisperModel(
        MODEL_NAME,
        device="cuda",
        compute_type=COMPUTE_TYPE,
        device_index=0,      # cambia si usas otra GPU
        num_workers=1        # hilos internos del modelo (no subir en GPU)
    )
    while True:
        audio_path = q.get()
        if audio_path is None:
            q.task_done()
            break
        try:
            msg = transcribe_with_model(model, audio_path, OUTPUT_DIR)
        except Exception as e:
            msg = f"❌ {audio_path.name} → {e}"
        with lock:
            results.append(msg)
            print(msg)
        q.task_done()

def main():
    files = list(gather_audio_files(INPUT_ROOT))
    total = len(files)
    if total == 0:
        print("⚠️ No se encontraron audios.")
        return

    # Ordenar por tamaño descendente para balancear carga (los largos primero)
    files.sort(key=lambda p: p.stat().st_size if p.exists() else 0, reverse=True)

    print(f"🗂️  Audios: {total}  ·  Workers(modelos): {MAX_MODELS}\n")

    q = Queue(maxsize=MAX_MODELS * 4)  # prefetched items
    results, lock = [], threading.Lock()
    threads = []

    # Lanzar N workers (cada uno con su modelo)
    for i in range(MAX_MODELS):
        t = threading.Thread(target=worker_loop, args=(i, q, results, lock), daemon=True)
        t.start()
        threads.append(t)

    # Ingesta en la cola con barra de progreso simple
    start = time.time()
    for idx, f in enumerate(files, 1):
        q.put(f)
        if idx % 50 == 0:
            elapsed = time.time() - start
            done = len(results)
            if done:
                rate = done / elapsed               # archivos/seg
                eta  = (total - done) / rate
                print(f"⏳ {done}/{total} · {rate*60:.1f} a/min · ETA {eta/60:.1f} min")

    # Señales de fin
    for _ in range(MAX_MODELS):
        q.put(None)

    q.join()  # esperar a que terminen
    
    dur = time.time() - start
    print(f"\n🏁 Listo: {len(results)}/{total} en {dur/60:.1f} min "
          f"({(total/(dur/60)):0.1f} audios/min aprox)")

if __name__ == "__main__":
    main()


## evaluacion 

In [ ]:
import os
import re
import json
import time
import hashlib
import unicodedata
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Optional, Tuple, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass

import pandas as pd
import requests
import pyodbc


# ───────────────────── RUTAS ─────────────────────
INPUT_TXT_DIR = Path(r"C:\descargas\ventas_audios_16-09-2025_hoy_12_19")
OUTPUT_DIR    = Path(r"C:\descargas\ventas_audios_16-09-2025_hoy_12_191")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

FECHA_ANALISIS = datetime.now().strftime("%Y-%m-%d")
TIPO = "llamada"

# ───────────────────── OLLAMA ─────────────────────
USE_OLLAMA = True
OLLAMA_BASE = "http://localhost:11434"
OLLAMA_MODEL = "qwen2.5:7b-instruct"
OLLAMA_TIMEOUT = 180
OLLAMA_MAX_RETRIES = 2
MAX_OLLAMA_WORKERS = 2  # 1-3 según tu PC

# LLM solo cuando aporta valor:
OLLAMA_ONLY_BORDERLINE = True
BORDERLINE_MIN = 0.45
BORDERLINE_MAX = 0.90

# Revisar humano si baja confianza del modelo:
REVIEW_MIN_CONF = 0.65

# Cache LLM:
CACHE_DIR = OUTPUT_DIR / "_ollama_cache"
CACHE_DIR.mkdir(exist_ok=True)

# ───────────────────── SQL (EMBEBIDO + AUTO-GUARDADO) ─────────────────────
SQL_SERVER   = "172.16.1.33"
SQL_DATABASE = "CUN_REPOSITORIO"
SQL_USER     = "haider_bello"

# Opción A (recomendada): dejar None para que lo pida 1 vez y lo guarde
SQL_PASSWORD: Optional[str] = None

# Opción B (NO recomendada): puedes hardcodear aquí (te lo dejo comentado)
# SQL_PASSWORD = r"TU_PASSWORD_AQUI"

SAVE_SQL_PASSWORD_LOCALLY = True
SQL_CONFIG_DIR = Path.home() / ".cun_call_eval"
SQL_CONFIG_DIR.mkdir(exist_ok=True)
SQL_CONFIG_FILE = SQL_CONFIG_DIR / "sql_config.json"


# ───────────────────── DICCIONARIOS (ETAPAS) ─────────────────────
diccionarios_keywords = {
    "saludo": sorted({
        "alo","hola","ola","ohla","buenos dias","buenas tardes","buenas noches",
        "saludo","bienvenido","me comunico","le saludo","le puedo ayudar",
        "en que le puedo servir","soy","me presento","habla con","hablo con",
        "muchas gracias","con gusto","hasta luego","que tenga un buen dia"
    }),
    "indagacion": sorted({
        "confirmar","verificar","ubicacion","ciudad","modalidad","programa","edad",
        "documento","icfes","homologacion","estudios previos","interes",
        "estas interesado","estas interesada","cuentame","cuéntame"
    }),
    "programas": sorted({
        "ingenieria","derecho","administracion","comunicacion social","periodismo",
        "contaduria","sistemas","industrial","salud","software","mercadeo","publicidad"
    }),
    "argumentacion": sorted({
        "beca","descuento","financiacion","convenio","virtual","presencial",
        "flexibilidad","salidas laborales","costo","valor semestre","cuotas",
        "requisitos","proceso","beneficios","homologar","credito educativo"
    }),
    "objecion": sorted({
        "muy caro","no tengo tiempo","lo pensare","lo pensaré","duda","que te preocupa",
        "entiendo","comprendo","opciones de financiacion","plan de pagos",
        "flexibilidad de horarios","estudia a tu ritmo"
    }),
    # cierre incluye también frases de seguimiento
    "cierre": sorted({
        "te inscribo","te llamo mañana","te llamo manana","te contacto mañana","te contacto manana",
        "agendamos llamada","agendemos","te confirmo cita","confirmo la cita","quedamos en",
        "confirmar inscripcion","enviar documentos","realizar pago","matricula","inscripcion",
        "te envio info","te envío info","conversemos por whatsapp","link","portal"
    }),
}

# Score por etapa: mínimo ocurrencias y peso
requisitos_keywords = {
    "saludo": (1, 1/6),
    "indagacion": (1, 1/6),
    "programas": (1, 1/6),
    "argumentacion": (1, 1/6),
    "objecion": (1, 1/6),
    "cierre": (1, 1/6),
}

# ───────────────────── GATILLOS PARA ENTENDER “RESULTADO” ─────────────────────
ANTI_CIERRE_FRASES = [
    "lo voy a pensar", "lo pensare", "lo pensaré", "tengo que pensarlo", "tengo que pensarlo mas",
    "no estoy seguro", "no estoy segura", "despues", "después", "mas adelante", "más adelante",
    "no hoy", "luego", "te aviso", "dejame revisar", "déjame revisar", "dejame consultarlo",
    "no puedo ahora", "en otro momento", "por ahora no"
]

# Cierre real (venta)
CIERRE_REAL_FRASES = [
    "ya pague", "ya pagué", "pago hoy", "voy a pagar hoy", "lo pago hoy", "ya hice el pago",
    "ya me inscribi", "ya me inscribí", "ya me matricule", "ya me matriculé",
    "quede inscrito", "quedé inscrito", "quedo inscrita", "quedó inscrita",
    "matricula realizada", "matrícula realizada"
]

# Seguimiento para cierre (tu criterio de efectiva)
SEGUIMIENTO_FRASES = [
    "te llamo mañana", "te llamo manana", "te marco mañana", "te marco manana",
    "te contacto mañana", "te contacto manana", "te vuelvo a llamar",
    "agendamos llamada", "agendemos", "queda agendada", "quedo agendada", "quedó agendada",
    "confirmo la cita", "te confirmo cita", "quedamos en", "nos hablamos", "nos hablamos mañana",
    "te envío la información", "te envio la informacion", "te envío el link", "te envio el link",
    "seguimiento", "seguimos", "continuamos"
]


# ───────────────────── UTILIDADES TEXTO ─────────────────────
def leer_txt(path: Path) -> str:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            return path.read_text(encoding=enc, errors="ignore")
        except Exception:
            continue
    return ""

def normalizar_texto(texto: str) -> str:
    if not isinstance(texto, str):
        return ""
    t = texto.lower()
    t = unicodedata.normalize("NFKD", t).encode("ascii", "ignore").decode("utf-8")
    t = re.sub(r"[^\w\s]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

def tail_text(texto_norm: str, ratio: float = 0.30) -> str:
    if not texto_norm:
        return ""
    n = len(texto_norm)
    return texto_norm[int(n * (1 - ratio)) :]

def _norm_token(s: str) -> str:
    s = s.lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    return re.sub(r"\s+", " ", s).strip()

def contiene_frase(texto_norm: str, frases: List[str]) -> bool:
    for f in frases:
        if _norm_token(f) in texto_norm:
            return True
    return False

def compilar_patrones(diccionarios: Dict[str, List[str]]) -> Dict[str, re.Pattern]:
    patrones = {}
    for cat, claves in diccionarios.items():
        claves_norm = sorted({_norm_token(c) for c in claves if c and str(c).strip()})
        if not claves_norm:
            patrones[cat] = re.compile(r"^\b$")
            continue
        altern = "|".join(re.escape(c) for c in claves_norm)
        patrones[cat] = re.compile(rf"(?<!\w)({altern})(?!\w)", flags=re.IGNORECASE)
    return patrones

def contar_por_categoria(texto_norm: str, patrones: Dict[str, re.Pattern]) -> Dict[str, int]:
    return {cat: len(pat.findall(texto_norm)) for cat, pat in patrones.items()}

def score_proceso_reglas(conteos: Dict[str, int]) -> Tuple[float, Dict[str, bool]]:
    etapas_ok = {}
    score = 0.0
    for cat, (minimo, peso) in requisitos_keywords.items():
        ok = conteos.get(cat, 0) >= minimo
        etapas_ok[cat] = ok
        if ok:
            score += peso
    return round(score, 3), etapas_ok


# ───────────────────── METADATOS DESDE NOMBRE ─────────────────────
def extraer_correo(nombre_archivo: str) -> str:
    cuerpo = Path(nombre_archivo).stem
    m = re.search(r'([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', cuerpo)
    return (m.group(1) if m else "").strip().lower()

def extraer_nombre_asesor(nombre_archivo: str) -> str:
    cuerpo = Path(nombre_archivo).stem
    m = re.search(r"by\s+(.+?)@", cuerpo, flags=re.IGNORECASE)
    bruto = m.group(1) if m else cuerpo
    limpio = re.sub(r"\s{2,}", " ", bruto.replace("_", " ").replace("-", " ").strip())
    return limpio.title() if limpio else ""

def extraer_celular(nombre_archivo: str) -> str:
    m = re.search(r'^(\+?\d{10,15})', Path(nombre_archivo).stem)
    return m.group(1) if m else ""


# ───────────────────── SQL PASSWORD AUTO (sin configurar nada) ─────────────────────
def load_sql_password() -> str:
    global SQL_PASSWORD

    if SQL_PASSWORD:
        return SQL_PASSWORD

    # si hay archivo guardado
    if SQL_CONFIG_FILE.exists():
        try:
            cfg = json.loads(SQL_CONFIG_FILE.read_text(encoding="utf-8"))
            pw = (cfg.get("SQL_PASSWORD") or "").strip()
            if pw:
                SQL_PASSWORD = pw
                return SQL_PASSWORD
        except Exception:
            pass

    # pedir una vez
    pw = getpass("🔐 Ingresa tu SQL_PASSWORD (se guardará localmente si está habilitado): ").strip()
    if not pw:
        raise RuntimeError("SQL_PASSWORD vacío. No puedo conectar a SQL Server.")
    SQL_PASSWORD = pw

    if SAVE_SQL_PASSWORD_LOCALLY:
        try:
            SQL_CONFIG_FILE.write_text(json.dumps({"SQL_PASSWORD": pw}, ensure_ascii=False, indent=2), encoding="utf-8")
            print(f"✅ Password guardado localmente en: {SQL_CONFIG_FILE}")
        except Exception as e:
            print(f"⚠️ No pude guardar el password localmente: {e}")

    return SQL_PASSWORD

def cargar_cedulas_por_correo() -> Dict[str, str]:
    mapping = {}
    pw = load_sql_password()

    conn = None
    try:
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'
            f'SERVER={SQL_SERVER};'
            f'DATABASE={SQL_DATABASE};'
            f'UID={SQL_USER};'
            f'PWD={pw};'
            'Trusted_Connection=no;'
        )
        query = """
            SELECT 
                CAST(Identificacion AS VARCHAR(64)) AS cedula,
                CAST(box_mail AS VARCHAR(320))      AS Correo
            FROM dbo.Planta_Activa
            WHERE box_mail IS NOT NULL AND LTRIM(RTRIM(box_mail)) <> ''
        """
        df_map = pd.read_sql(query, conn)
        df_map['Correo'] = df_map['Correo'].astype(str).str.strip().str.lower()
        df_map['cedula'] = df_map['cedula'].astype(str).str.strip()
        mapping = dict(zip(df_map['Correo'], df_map['cedula']))
        print(f"🗂️ Cédulas cargadas desde SQL: {len(mapping)} correos.")
    except Exception as e:
        print(f"⚠️ No se pudo consultar Planta_Activa: {e}")
    finally:
        try:
            if conn is not None:
                conn.close()
        except Exception:
            pass

    return mapping


# ───────────────────── CACHE ─────────────────────
def file_hash(path: Path) -> str:
    h = hashlib.sha1()
    h.update(str(path).encode("utf-8"))
    try:
        st = path.stat()
        h.update(str(st.st_mtime_ns).encode("utf-8"))
        h.update(str(st.st_size).encode("utf-8"))
    except Exception:
        pass
    return h.hexdigest()[:16]

def cache_path_for_txt(txt_path: Path) -> Path:
    return CACHE_DIR / f"{txt_path.stem}__{file_hash(txt_path)}.json"


# ───────────────────── OLLAMA ─────────────────────
def smart_trim_text(text: str, max_chars: int = 12000) -> str:
    text = (text or "").strip()
    if len(text) <= max_chars:
        return text
    head = text[: int(max_chars * 0.7)]
    tail = text[-int(max_chars * 0.3):]
    return head + "\n...\n" + tail

def _extract_json_loose(s: str) -> Optional[dict]:
    if not s:
        return None
    try:
        return json.loads(s)
    except Exception:
        pass
    m = re.search(r"\{.*\}", s, flags=re.DOTALL)
    if not m:
        return None
    try:
        return json.loads(m.group(0))
    except Exception:
        return None

def ollama_chat_json(system: str, user: str) -> dict:
    last_err = None

    chat_url = f"{OLLAMA_BASE}/api/chat"
    payload_chat = {
        "model": OLLAMA_MODEL,
        "messages": [{"role":"system","content":system},{"role":"user","content":user}],
        "stream": False,
        "format": "json",
        "options": {"temperature": 0.1, "num_ctx": 8192}
    }

    for attempt in range(1, OLLAMA_MAX_RETRIES + 1):
        try:
            r = requests.post(chat_url, json=payload_chat, timeout=OLLAMA_TIMEOUT)
            if r.status_code != 200:
                raise RuntimeError(f"chat HTTP {r.status_code}: {r.text[:250]}")
            data = r.json()
            content = (data.get("message") or {}).get("content", "")
            parsed = _extract_json_loose(content)
            if not parsed:
                raise ValueError(f"No parse JSON (chat). Resp: {content[:200]}")
            return parsed
        except Exception as e:
            last_err = e
            time.sleep(1.2 * attempt)

    gen_url = f"{OLLAMA_BASE}/api/generate"
    payload_gen = {
        "model": OLLAMA_MODEL,
        "prompt": system + "\n\n" + user,
        "stream": False,
        "format": "json",
        "options": {"temperature": 0.1, "num_ctx": 8192}
    }
    for attempt in range(1, OLLAMA_MAX_RETRIES + 1):
        try:
            r = requests.post(gen_url, json=payload_gen, timeout=OLLAMA_TIMEOUT)
            if r.status_code != 200:
                raise RuntimeError(f"generate HTTP {r.status_code}: {r.text[:250]}")
            data = r.json()
            content = data.get("response", "")
            parsed = _extract_json_loose(content)
            if not parsed:
                raise ValueError(f"No parse JSON (generate). Resp: {content[:200]}")
            return parsed
        except Exception as e:
            last_err = e
            time.sleep(1.2 * attempt)

    raise RuntimeError(f"Ollama falló: {last_err}")


# ───────────────────── ¿CUÁNDO LLAMAR AL LLM? ─────────────────────
def should_call_ollama(score_proc: float, texto_norm: str, conteos: Dict[str, int]) -> Tuple[bool, str]:
    ttail = tail_text(texto_norm, ratio=0.30)

    anti = contiene_frase(ttail, ANTI_CIERRE_FRASES)
    hay_cierre = conteos.get("cierre", 0) > 0 or contiene_frase(ttail, ["pago", "matricula", "inscripcion", "portal", "link"])
    hay_seguimiento = contiene_frase(ttail, SEGUIMIENTO_FRASES)

    # Si hay indecisión al final pero también hay cierre/seguimiento mencionado => el LLM aclara si quedó seguimiento real
    if anti and (hay_cierre or hay_seguimiento):
        return True, "contradiccion_indecision_vs_cierre_seguimiento"

    # anti-cierre al final => LLM decide si se salvó con seguimiento
    if anti:
        return True, "anti_cierre_en_final"

    # si hay seguimiento/cierre pero ambiguo => LLM ayuda (muchas veces el diccionario marca cierre pero no está confirmado)
    if (hay_cierre or hay_seguimiento) and score_proc >= 0.75:
        return True, "validar_resultado_cierre_o_seguimiento"

    # borderline
    if OLLAMA_ONLY_BORDERLINE and (BORDERLINE_MIN <= score_proc <= BORDERLINE_MAX):
        return True, "borderline_score"

    if not OLLAMA_ONLY_BORDERLINE:
        return True, "modo_llm_todos"

    return False, "no_aplica"


# ───────────────────── LLM: EVALUAR PROCESO + RESULTADO (CIERRE O SEGUIMIENTO) ─────────────────────
def evaluar_llm(texto_original: str, conteos: Dict[str, int], score_proc: float, etapas_ok: Dict[str, bool]) -> Dict[str, Any]:
    system = (
        "Eres auditor experto de calidad de llamadas de admisiones (CUN). "
        "Evalúas PROCESO (etapas) y RESULTADO (CIERRE o SEGUIMIENTO para cerrar). "
        "Responde SOLO en JSON válido, sin markdown, sin texto adicional."
    )

    user_obj = {
        "definiciones": {
            "proceso": (
                "Etapas: saludo/presentación, indagación (necesidad+datos), programas/fit, "
                "argumentación (valor+costos/financiación), objeciones (si aparecen) y cierre."
            ),
            "resultado": (
                "La llamada es EFECTIVA si hay: "
                "(A) CIERRE de venta (matrícula/inscripción/pago confirmado o intención explícita inmediata) "
                "o (B) SEGUIMIENTO claro para cerrar (próxima llamada/agendamiento/compromiso de contacto) con consentimiento del cliente. "
                "Si el cliente termina en 'lo pienso' sin seguimiento pactado, entonces NO hay resultado."
            )
        },
        "datos_reglas": {
            "conteos_diccionarios": conteos,
            "score_proceso_reglas": score_proc,
            "etapas_ok_reglas": etapas_ok
        },
        "transcripcion": smart_trim_text(texto_original, 12000),
        "salida_esperada": {
            "proceso": {
                "etapas": {
                    "saludo": {"presente": True, "evidencia": "frase literal corta"},
                    "indagacion": {"presente": True, "evidencia": "frase literal corta"},
                    "programas": {"presente": True, "evidencia": "frase literal corta"},
                    "argumentacion": {"presente": True, "evidencia": "frase literal corta"},
                    "objecion": {"presente": True, "evidencia": "frase literal corta"},
                    "cierre": {"presente": True, "evidencia": "frase literal corta"}
                }
            },
            "resultado": {
                "tipo": "cierre|seguimiento|ninguno",
                "confirmado": True,
                "evidencia": "frase literal corta"
            },
            "score_resultado": 0.0,
            "efectiva": 0,
            "sentimiento_cliente": "positivo|neutro|negativo",
            "confianza": 0.0,
            "razones": ["max 5"],
            "mejoras": ["max 5"]
        }
    }

    parsed = ollama_chat_json(system=system, user=json.dumps(user_obj, ensure_ascii=False))

    # saneo
    parsed.setdefault("score_resultado", 0.0)
    parsed.setdefault("confianza", 0.5)
    parsed.setdefault("sentimiento_cliente", "neutro")
    parsed.setdefault("razones", [])
    parsed.setdefault("mejoras", [])
    parsed.setdefault("resultado", {})
    parsed.setdefault("proceso", {})
    parsed.setdefault("efectiva", 0)

    try:
        parsed["score_resultado"] = float(parsed.get("score_resultado", 0.0))
    except Exception:
        parsed["score_resultado"] = 0.0

    try:
        parsed["confianza"] = float(parsed.get("confianza", 0.5))
    except Exception:
        parsed["confianza"] = 0.5

    try:
        parsed["efectiva"] = int(parsed.get("efectiva", 0))
    except Exception:
        parsed["efectiva"] = 0

    res = parsed.get("resultado") or {}
    if not isinstance(res, dict):
        res = {}
    res.setdefault("tipo", "ninguno")
    res.setdefault("confirmado", False)
    res.setdefault("evidencia", "")
    parsed["resultado"] = res

    return parsed


# ───────────────────── RESULTADO SIN LLM (CONSERVADOR) ─────────────────────
def estimar_score_resultado_sin_llm(texto_norm: str) -> Tuple[float, str]:
    ttail = tail_text(texto_norm, ratio=0.30)
    anti = contiene_frase(ttail, ANTI_CIERRE_FRASES)

    cierre_real = contiene_frase(ttail, CIERRE_REAL_FRASES)
    seguimiento = contiene_frase(ttail, SEGUIMIENTO_FRASES)

    # Si hay anti-cierre pero hay seguimiento pactado => resultado medio (porque sí hay plan de cierre)
    if anti and seguimiento:
        return 0.65, "regla_seguimiento_a_pesar_de_indecision"

    if cierre_real and not anti:
        return 0.85, "regla_cierre_real"

    if seguimiento and not anti:
        return 0.70, "regla_seguimiento"

    if anti:
        return 0.15, "regla_anti_cierre"

    return 0.30, "regla_sin_evidencia_resultado"


# ───────────────────── ETIQUETA FINAL (TU DEFINICIÓN) ─────────────────────
def etiqueta_final(score_proceso: float, score_resultado: float) -> str:
    # efectiva = proceso sólido + resultado (cierre o seguimiento)
    if score_proceso >= 0.75 and score_resultado >= 0.70:
        return "efectiva"
    # buena pero no cerrada = buena ejecución sin resultado suficiente
    if score_proceso >= 0.75 and score_resultado < 0.70:
        return "buena_no_cerrada"
    return "no_efectiva"


# ───────────────────── ANALIZAR 1 TXT ─────────────────────
def analizar_un_txt(p: Path, patrones: Dict[str, re.Pattern], cedulas_sql: Dict[str, str]) -> Dict[str, Any]:
    texto = leer_txt(p)
    texto_norm = normalizar_texto(texto)

    conteos = contar_por_categoria(texto_norm, patrones)
    score_proc, etapas_ok = score_proceso_reglas(conteos)

    correo  = extraer_correo(p.name)
    asesor  = extraer_nombre_asesor(p.name)
    celular = extraer_celular(p.name)
    cedula  = cedulas_sql.get(correo, "") if correo else ""

    tamano_B  = p.stat().st_size if p.exists() else 0
    palabras  = len(texto_norm.split()) if texto_norm else 0
    oraciones = len(re.findall(r"[.!?]+", texto)) if texto else 0

    llm_usado = False
    llm_motivo = ""
    llm_conf = ""
    llm_sent = ""
    llm_razones = ""
    llm_mejoras = ""
    res_tipo = ""
    res_conf = ""
    res_evid = ""
    evidencias_etapas = {}

    score_res = 0.0
    res_fuente = "regla"
    revisar_humano = False

    if USE_OLLAMA:
        call, motivo = should_call_ollama(score_proc, texto_norm, conteos)
        if call:
            llm_usado = True
            llm_motivo = motivo

            cpath = cache_path_for_txt(p)
            llm = None
            if cpath.exists():
                try:
                    llm = json.loads(cpath.read_text(encoding="utf-8"))
                except Exception:
                    llm = None

            if llm is None:
                llm = evaluar_llm(texto_original=texto, conteos=conteos, score_proc=score_proc, etapas_ok=etapas_ok)
                try:
                    cpath.write_text(json.dumps(llm, ensure_ascii=False, indent=2), encoding="utf-8")
                except Exception:
                    pass

            score_res = float(llm.get("score_resultado", 0.0))
            res_fuente = "llm"

            llm_conf = llm.get("confianza", "")
            llm_sent = llm.get("sentimiento_cliente", "")
            llm_razones = "; ".join((llm.get("razones") or [])[:5])
            llm_mejoras = "; ".join((llm.get("mejoras") or [])[:5])

            res = llm.get("resultado") or {}
            res_tipo = str(res.get("tipo", "ninguno"))
            res_conf = str(bool(res.get("confirmado", False)))
            res_evid = str(res.get("evidencia", ""))[:220]

            proc = llm.get("proceso") or {}
            etapas = (proc.get("etapas") if isinstance(proc, dict) else {}) or {}
            for k in ["saludo","indagacion","programas","argumentacion","objecion","cierre"]:
                v = etapas.get(k, {}) if isinstance(etapas, dict) else {}
                if isinstance(v, dict):
                    evidencias_etapas[k] = {
                        "presente": bool(v.get("presente", False)),
                        "evidencia": str(v.get("evidencia", ""))[:180]
                    }

            # marcar revisión humana por baja confianza
            try:
                if float(llm_conf) < REVIEW_MIN_CONF:
                    revisar_humano = True
            except Exception:
                revisar_humano = True

        else:
            score_res, res_fuente = estimar_score_resultado_sin_llm(texto_norm)

    else:
        score_res, res_fuente = estimar_score_resultado_sin_llm(texto_norm)

    label = etiqueta_final(score_proc, score_res)

    row = {
        "correo": correo,
        "cedula_asesor": cedula,
        "asesor": asesor,
        "asesor_corto": asesor,
        "fecha": FECHA_ANALISIS,

        "saludo": conteos.get("saludo", 0),
        "indagacion": conteos.get("indagacion", 0),
        "programas": conteos.get("programas", 0),
        "argumentacion": conteos.get("argumentacion", 0),
        "objecion": conteos.get("objecion", 0),
        "cierre": conteos.get("cierre", 0),

        "score_proceso": score_proc,
        "score_resultado": round(float(score_res), 3),
        "resultado_fuente": res_fuente,

        "etiqueta_final": label,
        "revisar_humano": "1" if revisar_humano else "0",

        "llm_usado": "1" if llm_usado else "0",
        "llm_motivo": llm_motivo,
        "llm_confianza": llm_conf,
        "llm_sentimiento_cliente": llm_sent,

        "resultado_tipo": res_tipo,
        "resultado_confirmado": res_conf,
        "resultado_evidencia": res_evid,

        "llm_razones": llm_razones,
        "llm_mejoras": llm_mejoras,

        "palabras": palabras,
        "oraciones": oraciones,
        "archivo": p.name,
        "tamano_B": tamano_B,
        "celular": celular,
        "tipo": TIPO,
    }

    for k in ["saludo","indagacion","programas","argumentacion","objecion","cierre"]:
        if k in evidencias_etapas:
            row[f"llm_etapa_{k}"] = str(evidencias_etapas[k].get("presente", False))
            row[f"llm_evid_{k}"] = evidencias_etapas[k].get("evidencia", "")
        else:
            row[f"llm_etapa_{k}"] = ""
            row[f"llm_evid_{k}"] = ""

    return row


# ───────────────────── MAIN ─────────────────────
def analizar_txts_y_exportar():
    cedulas_sql = cargar_cedulas_por_correo()

    txt_files = list(INPUT_TXT_DIR.rglob("*.txt"))
    if not txt_files:
        raise RuntimeError(f"No encontré .txt en: {INPUT_TXT_DIR}")

    patrones = compilar_patrones(diccionarios_keywords)

    workers = MAX_OLLAMA_WORKERS if USE_OLLAMA else min(8, os.cpu_count() or 4)

    filas = []
    start = time.time()

    with ThreadPoolExecutor(max_workers=workers) as ex:
        futs = [ex.submit(analizar_un_txt, p, patrones, cedulas_sql) for p in txt_files]
        for i, f in enumerate(as_completed(futs), 1):
            try:
                filas.append(f.result())
            except Exception as e:
                filas.append({"archivo": "ERROR", "llm_razones": str(e)})
            if i % 50 == 0:
                print(f"Procesados: {i}/{len(txt_files)}")

    df = pd.DataFrame(filas)

    cols = [
        "correo","cedula_asesor","asesor","asesor_corto","fecha",
        "saludo","indagacion","programas","argumentacion","objecion","cierre",
        "score_proceso","score_resultado","resultado_fuente",
        "etiqueta_final","revisar_humano",
        "llm_usado","llm_motivo","llm_confianza","llm_sentimiento_cliente",
        "resultado_tipo","resultado_confirmado","resultado_evidencia",
        "llm_razones","llm_mejoras",
        "palabras","oraciones","archivo","tamano_B","celular","tipo",
        "llm_etapa_saludo","llm_evid_saludo",
        "llm_etapa_indagacion","llm_evid_indagacion",
        "llm_etapa_programas","llm_evid_programas",
        "llm_etapa_argumentacion","llm_evid_argumentacion",
        "llm_etapa_objecion","llm_evid_objecion",
        "llm_etapa_cierre","llm_evid_cierre",
    ]
    for c in cols:
        if c not in df.columns:
            df[c] = ""
    df = df[cols]

    base = f"Ventas_Evaluacion_Robusta_{datetime.now().strftime('%Y-%m-%d')}"
    xlsx = OUTPUT_DIR / f"{base}.xlsx"
    csv  = OUTPUT_DIR / f"{base}.csv"

    try:
        df.to_excel(xlsx, index=False)
        print(f"✅ Excel generado: {xlsx}")
    except Exception as e:
        print(f"⚠️ No se pudo escribir XLSX ({e}). Guardando CSV…")
        df.to_csv(csv, index=False, encoding="utf-8-sig")
        print(f"✅ CSV generado: {csv}")

    dt = round(time.time() - start, 1)
    print("\nResumen etiqueta_final:")
    print(df["etiqueta_final"].value_counts(dropna=False))
    print(f"\n⏱️ Tiempo: {dt}s | Archivos: {len(df)} | Cache: {CACHE_DIR}")
    return df


if __name__ == "__main__":
    analizar_txts_y_exportar()
